In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import os, shutil
import zipfile

In [ ]:
zip_ref = zipfile.ZipFile('../input/dogs-vs-cats/train.zip', 'r')
zip_ref.extractall('train')
zip_ref.close()

zip_ref = zipfile.ZipFile('../input/dogs-vs-cats/test1.zip', 'r')
zip_ref.extractall('test')
zip_ref.close()

In [ ]:
train_full_dir = './train/train/'
test_full_dir = './test/test1/'

create_if_not = ['training/dogs/','training/cats/','testing/']
for i in create_if_not:
    if os.path.isdir(i) == 0:
        os.makedirs(i)

In [ ]:
train_filenames = os.listdir(train_full_dir)
categories = []
for tr_filename in train_filenames:
    category = tr_filename.split('.')[0]
    if category == 'dog':
        shutil.move(train_full_dir+ tr_filename, 'training/dogs/'+ tr_filename)
    else:
        shutil.move(train_full_dir+ tr_filename, 'training/cats/'+ tr_filename)

test_filenames = os.listdir(test_full_dir)
for t_filename in test_filenames:
        shutil.move(test_full_dir+ t_filename, 'testing/'+ t_filename)
        
shutil.rmtree('./train')
shutil.rmtree('./test')

In [ ]:
print("Training Dog images: ",len(os.listdir('./training/dogs')))
print("Training Cat images: ",len(os.listdir('./training/cats')))
print("Testing images: ",len(os.listdir('./testing')))

In [ ]:
image_size = (180,180)
batch_size = 50

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './training',
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './training',
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
class_names = train_ds.class_names
class_names

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomRotation(0.1),])

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
input_shape = image_size + (3,)
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(previous_block_activation)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

In [ ]:
model = make_model(input_shape=image_size + (3,), num_classes=2)

In [ ]:
epochs = 25

callbacks = [keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5")]
model.compile(optimizer=keras.optimizers.Adam(1e-3),loss="binary_crossentropy",metrics=["acc"])
model.fit(train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds)

In [ ]:
model.save('./catsvsdogs - advanced.h5')

In [ ]:
def display_training_curves(training, validation, title, subplot):
    ax = plt.subplot(subplot)
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    ax.set_xlabel('epoch')
    ax.legend(['training', 'validation'])

plt.subplots(figsize=(10,10))
plt.tight_layout()
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 211)
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 212)

In [ ]:
import random
test_filenames = os.listdir('./testing')
rand = random.choice(test_filenames)
image_path = './testing/' + rand
img = keras.preprocessing.image.load_img(image_path, target_size=(180, 180))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


from PIL import Image
Image.open(image_path)  